In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/mina-lou/Desktop/Frank')
import isingtebd 

In [36]:
Nx=3
test_MPS = isingtebd.Initial_state(Nx)

In [37]:
print(test_MPS.Bs) 

[array([[[1.],
        [0.]]]), array([[[1.],
        [0.]]]), array([[[1.],
        [0.]]])]


In [58]:
J=1
T=10
dt=1 
chi_m=30
eps=1

model_test = isingtebd.Ising_model(Nx, J, T, dt, chi_m, eps)
updated_state = isingtebd.update_bond(test_MPS,model_test,2) 

theta =   [[[[1.]
   [0.]]

  [[0.]
   [0.]]]] thetashape =  (1, 2, 2, 1)


IndexError: list index out of range

In [49]:
WS = [1, 1, 1, 1, 1 ]
len(WS)

5